# census onehot PROBLEM
onehot: MCAR은 정상 작동, MAR, MNAR train도 가능 but evaluate할 때 코드가 돌아가지 않음. 원인은 tensor size가 맞지 않음.
- main_model_table.py의 evaluate에서 문제 발생
- batch에 대해 process_data(TabCSDI 클래스에 선언) 했을 때 문제가 생기는지. 
- evaluate_onehot에 문제가 있나? 근데 MCAR일 때는 왜 아무런 문제가 없냐고.
- What to demonstrate: 1. Onehot - MCAR과의 차이 2. FT, Analog와의 차이
- 207 = 15 + 96 + 96 = 111 + 96

In [2]:
import pickle
import yaml
import os
import math
import re
import numpy as np
import pandas as pd
import category_encoders as ce
from torch.utils.data import DataLoader, Dataset
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
# import required packages
import torch
# import wget
# wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')
from utils import *

In [3]:
import argparse
import torch
import datetime
import json
import yaml
import os

from src.main_model_table import TabCSDI
from src.utils_table import train, evaluate_onehot
from dataset_census.dataset_census_onehot2 import get_dataloader

In [4]:
modelfolder = "census_onehot_fold5/MAR0.4_20240225_070242/"
testmissingratio = 0.2
config = "census_onehot_analog.yaml"
# device = "cuda"
if torch.cuda.is_available() : device = torch.device('cuda')
# elif torch.backends.mps.is_available() : device = torch.device('mps')
else : device=torch.device('cpu')
print(f'Using {device}')
exe_name = "census"

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

path = "config/" + config
with open(path, "r") as f:
    config = yaml.safe_load(f)
config["model"]["is_unconditional"] = 0
config["model"]["test_missing_ratio"] = testmissingratio
model = TabCSDI(config, device).to(device)

Using cpu


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [31]:
train_loader, valid_loader, test_loader = get_dataloader(
    seed=2,
    nfold=5,
    batch_size=64,
    missing_ratio=0.4,
    mecha="MAR",
    opt="logistic",
    p_obs=0.5
)

--------Dataset created--------
Dataset size:20000 entries
--------------Dataset has not been normalized yet. Perform data normalization and store the mean value of each column.--------------
--------------Max-value for cont-variable column [9.000000e+01 1.484705e+06 1.600000e+01 9.999900e+04 4.356000e+03
 9.900000e+01]--------------
--------------Min-value for cont-variable column [1.7000e+01 1.9302e+04 1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00]--------------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
Training dataset size: 16000
Validation dataset size: 0
Testing dataset size: 4000


In [32]:
from tqdm import tqdm
torch.manual_seed(0);np.random.seed(0)
with torch.no_grad():
    with tqdm(test_loader, mininterval=5.0, maxinterval=50.0) as it:
        for batch_no, test_batch in enumerate(it, start=1):
            print(batch_no, test_batch)
            print(test_batch.keys())
            if batch_no == 3:
                break

  3%|▎         | 2/63 [00:00<00:00, 90.65it/s]

1 {'observed_data': tensor([[ 0.0541,  0.1763,  0.0625,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.0946,  0.1135,  0.8125,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.4865,  0.0929,  0.5625,  ..., -1.0000,  1.0000, -1.0000],
        ...,
        [ 0.1757,  0.0459,  0.1250,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.2973,  0.0968,  0.6250,  ..., -1.0000,  1.0000, -1.0000],
        [ 0.3108,  0.1174,  0.8125,  ..., -1.0000, -1.0000,  1.0000]],
       dtype=torch.float64), 'observed_mask': tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]]), 'gt_mask': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 1.

In [33]:
print(test_batch['observed_mask'].shape)
print(test_batch['gt_mask'].shape)
## 아..???
## test_loader 생성 시 gt_mask를 만들 때 무슨 문제가 있나. 왜 하필 Onehot, MAR만 이러나. 

torch.Size([64, 111])
torch.Size([64, 111])


In [34]:
# model.evaluate 풀어서 쓰기
# process_data
batch = test_batch
observed_data = batch["observed_data"][:, np.newaxis, :].to(device).float()
observed_mask = batch["observed_mask"][:, np.newaxis, :].to(device).float()
observed_tp = batch["timepoints"].to(device).float()
gt_mask = batch["gt_mask"][:, np.newaxis, :].to(device).float()
cut_length = torch.zeros(len(observed_data)).long().to(device)
for_pattern_mask = observed_mask

In [35]:
# evaluate
print(gt_mask.shape)
print(observed_mask.shape)

torch.Size([64, 1, 111])
torch.Size([64, 1, 111])


In [10]:
with torch.no_grad():
    cond_mask = gt_mask
    target_mask = observed_mask - cond_mask
    print(cond_mask.shape, target_mask.shape)

RuntimeError: The size of tensor a (111) must match the size of tensor b (207) at non-singleton dimension 2

그렇다면 데이터셋을 처리해 gt_mask를 생성하는 process_func를 살펴보자.

In [16]:
def produce_NA(X, p_miss, mecha="MAR", opt=None, p_obs=None, q=None):
    
    to_torch = torch.is_tensor(X)
    
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    np.random.seed(0);torch.manual_seed(0)
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    # X_nas = X.clone()
    # X_nas[mask.bool()] = np.nan
    
    # return X_nas.double() # tensor type
    return mask

In [11]:
path = './data_census_onehot/adult_trim.data'
missing_ratio = 0.4
encode = True
p_obs = 0.5
mecha = 'MAR'
opt = 'logistic'
cat_list = [1, 3, 5, 6, 7, 8, 9, 13, 14]

data = pd.read_csv(path, header=None)
data.replace(" ?", np.nan, inplace=True)

temp_list = [i for i in range(data.shape[1]) if i not in cat_list]
temp_list.extend(cat_list)
new_cols_order = temp_list
data = data.reindex(columns=data.columns[new_cols_order])
data.columns = [i for i in range(data.shape[1])]

# create two lists to store position
cont_list = [i for i in range(0, data.shape[1] - len(cat_list))]
cat_list = [i for i in range(len(cont_list), data.shape[1])]

observed_values = data.values
observed_masks = ~pd.isnull(data)
observed_masks = observed_masks.values

In [12]:
encoder2 = ce.ordinal.OrdinalEncoder(cols=data.columns[cat_list]) ## ordinal encoding
encoder2.fit(data)
new_df2 = encoder2.transform(data)
new_observed_values2 = new_df2.values

In [17]:
gt_masks = produce_NA(new_observed_values2, p_miss=missing_ratio, p_obs=p_obs, mecha=mecha, opt=opt)
gt_masks = np.array(gt_masks)

In [18]:
gt_masks.shape

(20000, 15)

In [19]:
encoder = ce.one_hot.OneHotEncoder(cols=data.columns[cat_list]) ## one hot encoding
encoder.fit(data)
new_df = encoder.transform(data)

# we now need to transform these masks to the new one, suitable for mixed data types.
cum_num_bits = 0
new_observed_masks = observed_masks.copy()
new_gt_masks = gt_masks.copy()

for index, col in enumerate(cat_list):
    corresponding_cols = len(
        [
            s
            for s in new_df.columns
            if isinstance(s, str) and s.startswith(str(col) + "_")
        ]
    )
    add_col_num = corresponding_cols
    insert_col_obs = observed_masks[:, col] ## masked columns(variables)
    insert_col_gt = gt_masks[:, col]

    for i in range(add_col_num - 1): ## transform masked columns into the form of one hot encoded columns
        new_observed_masks = np.insert(
            new_observed_masks, cum_num_bits + col, insert_col_obs, axis=1
        )
        new_gt_masks = np.insert(
            new_gt_masks, cum_num_bits + col, insert_col_gt, axis=1
        )
    cum_num_bits += add_col_num - 1

In [20]:
new_observed_values = new_df.values
new_observed_values_cat = new_observed_values[:, len(cont_list) :]
index = new_observed_values_cat == 0
new_observed_values_cat[index] = -1
new_observed_values[:, len(cont_list) :] = new_observed_values_cat
new_observed_values = np.nan_to_num(new_observed_values)
new_observed_values = new_observed_values.astype(np.float)

/var/folders/kk/41dsh9496q37sbz08xw3rgkr0000gn/T/ipykernel_67091/570892538.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new_observed_values = new_observed_values.astype(np.float)


In [23]:
print(new_gt_masks.shape)
print(new_observed_masks.shape)
# process_func은 문제 없음

(20000, 111)
(20000, 111)


In [24]:
# get_dataloader
from dataset_census.dataset_census_onehot2 import tabular_dataset

dataset = tabular_dataset(missing_ratio=0.4, seed=1, p_obs=0.5, mecha='MAR', opt='logistic')
print(dataset)

--------Normalized dataset loaded--------


In [26]:
indlist = np.arange(len(dataset))

np.random.seed(2)
np.random.shuffle(indlist)

tmp_ratio = 1/5
start = (int)(4 * len(dataset) * tmp_ratio)
end = start + int(len(dataset) * tmp_ratio)
test_index = indlist[start:end]
remain_index = np.delete(indlist, np.arange(start, end))

np.random.shuffle(remain_index)
num_train = (int)(len(remain_index)*1)
train_index = remain_index[:num_train]
valid_index = remain_index[num_train:]

In [30]:
indlist

array([  836,  1849, 16673, ..., 11798,  6637,  2575])

## 결론: 예전에 데이터 생성 함수를 잘못 만들었었는데, 그때의 시드로 만들어진 데이터가 계속 사용되고 있었음!

In [36]:
# 그래도 다시 확인
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
          return super().find_class(module, name)

In [39]:
with open("./data_census_onehot/missing_ratio-0.4_seed-1.pk", 'rb') as f :
    oh_data = CPU_Unpickler(f).load()

In [40]:
print(oh_data[0].shape)
print(oh_data[1].shape)
print(oh_data[2].shape)

(20000, 111)
(20000, 111)
(20000, 207)


## .pk 만들어보까

In [41]:
train_loader, valid_loader, test_loader = get_dataloader(
    seed=1,
    nfold=5,
    batch_size=64,
    missing_ratio=0.4,
    mecha="MAR",
    opt="logistic",
    p_obs=0.5
)

--------Dataset created--------
Dataset size:20000 entries
--------------Dataset has not been normalized yet. Perform data normalization and store the mean value of each column.--------------
--------------Max-value for cont-variable column [9.000000e+01 1.484705e+06 1.600000e+01 9.999900e+04 4.356000e+03
 9.900000e+01]--------------
--------------Min-value for cont-variable column [1.7000e+01 1.2285e+04 1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00]--------------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
Training dataset size: 16000
Validation dataset size: 0
Testing dataset size: 4000


# Checking dataframes

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data_census_ft/adult_trim.data',header=None)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,36,Self-emp-not-inc,89622,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,80,United-States,>50K
19996,34,Private,202498,12th,8,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,Dominican-Republic,<=50K
19997,72,Private,268861,7th-8th,4,Widowed,Other-service,Not-in-family,White,Female,0,0,99,?,<=50K
19998,54,Private,343242,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,44,United-States,>50K


In [3]:
# process_func in dataset_insurance_ft2.py
cat_list = [1,3,5,6,7,8,9,13,14]
# Swap columns
temp_list = [i for i in range(data.shape[1]) if i not in cat_list]
temp_list.extend(cat_list)
new_cols_order = temp_list
data = data.reindex(columns=data.columns[new_cols_order])
data.columns = [i for i in range(data.shape[1])]

# create two lists to store position
cont_list = [i for i in range(0, data.shape[1] - len(cat_list))]
cat_list = [i for i in range(len(cont_list), data.shape[1])]

observed_values = data.values
observed_masks = ~pd.isnull(data)
observed_masks = observed_masks.values

In [4]:
!pip install category_encoders


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
# This script is for generating .pk file for mixed data types dataset
import pickle
import yaml
import os
import math
import re
import numpy as np
import pandas as pd
import category_encoders as ce

from torch.utils.data import DataLoader, Dataset

import ssl

ssl._create_default_https_context = ssl._create_unverified_context
# import required packages
import torch
# import wget
# wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')
from utils import *

In [6]:
def produce_NA(X, p_miss, mecha="MAR", opt=None, p_obs=None, q=None):
    
    to_torch = torch.is_tensor(X)
    
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    np.random.seed(0);torch.manual_seed(0)
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    # X_nas = X.clone()
    # X_nas[mask.bool()] = np.nan
    
    # return X_nas.double() # tensor type
    return mask

In [7]:
encoder = ce.ordinal.OrdinalEncoder(cols=data.columns[cat_list])
encoder.fit(data)
new_df = encoder.transform(data)
new_observed_values = new_df.values
print(new_observed_values)

missing_ratio=0.2
p_obs=None
mecha='MCAR'
opt=None
gt_masks = produce_NA(new_observed_values, p_miss=missing_ratio, p_obs=p_obs, mecha=mecha, opt=opt)
gt_masks = np.array(gt_masks)

[[    39  77516     13 ...      1      1      1]
 [    50  83311     13 ...      1      1      1]
 [    38 215646      9 ...      1      1      1]
 ...
 [    72 268861      4 ...      2      5      1]
 [    54 343242     10 ...      1      1      2]
 [    30 460408      9 ...      1      1      2]]


In [8]:
gt_masks.shape

(20000, 15)

In [25]:
pd.DataFrame(new_observed_values)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,77516,13,2174,0,40,1,1,1,1,1,1,1,1,1
1,50,83311,13,0,0,13,2,1,2,2,2,1,1,1,1
2,38,215646,9,0,0,40,3,2,3,3,1,1,1,1,1
3,53,234721,7,0,0,40,3,3,2,3,2,2,1,1,1
4,28,338409,13,0,0,40,3,1,2,4,3,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,36,89622,9,0,0,80,2,2,2,7,2,1,1,1,2
19996,34,202498,8,0,0,40,3,16,2,5,2,1,1,25,1
19997,72,268861,4,0,0,99,3,9,7,5,1,1,2,5,1
19998,54,343242,10,0,0,44,3,6,2,2,2,1,1,1,2


In [27]:
# for insurance data
data = pd.read_csv('./data_insurance/insurance.csv',header=None)
# process_func in dataset_insurance_ft2.py
cat_list = [1,4,5]
# Swap columns
temp_list = [i for i in range(data.shape[1]) if i not in cat_list]
temp_list.extend(cat_list)
new_cols_order = temp_list
data = data.reindex(columns=data.columns[new_cols_order])
data.columns = [i for i in range(data.shape[1])]

# create two lists to store position
cont_list = [i for i in range(0, data.shape[1] - len(cat_list))]
cat_list = [i for i in range(len(cont_list), data.shape[1])]

observed_values = data.values
observed_masks = ~pd.isnull(data)
observed_masks = observed_masks.values

In [28]:
encoder = ce.ordinal.OrdinalEncoder(cols=data.columns[cat_list])
encoder.fit(data)
new_df = encoder.transform(data)
new_observed_values = new_df.values
print(new_observed_values)

gt_masks = produce_NA(new_observed_values, p_miss=missing_ratio, p_obs=p_obs, mecha=mecha, opt=opt)
gt_masks = np.array(gt_masks)

[[19.   27.9   0.   ...  1.    1.    1.  ]
 [18.   33.77  1.   ...  2.    2.    2.  ]
 [28.   33.    3.   ...  2.    2.    2.  ]
 ...
 [18.   36.85  0.   ...  1.    2.    2.  ]
 [21.   25.8   0.   ...  1.    2.    1.  ]
 [61.   29.07  0.   ...  1.    1.    3.  ]]


In [16]:
pd.DataFrame(new_observed_values)


,0,1,2,3,4,5,6
0,19,27.9,southwest,16884.924,1,1,1
1,18,33.77,southeast,1725.5523,2,2,2
2,28,33.0,southeast,4449.462,2,3,2
3,33,22.705,northwest,21984.47061,2,1,2
4,32,28.88,northwest,3866.8552,2,1,2
...,...,...,...,...,...,...,...
1333,50,30.97,northwest,10600.5483,2,3,2
1334,18,31.92,northeast,2205.9808,1,1,2
1335,18,36.85,southeast,1629.8335,1,1,2
1336,21,25.8,southwest,2007.945,1,1,2
